<a href="https://colab.research.google.com/github/rondinell/Intelig-ncia-Artificial/blob/main/artigo14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ====================================================================
# Parte 1: INSTALAÇÃO E IMPORTAÇÕES
# ====================================================================
print("Instalando e importando bibliotecas...")
!pip install yfinance mplfinance --quiet # Removendo biblioteca  ta para no futuro calcular de forma manual

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from datetime import date
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, LSTM, concatenate
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping

print("Bibliotecas carregadas com sucesso!")


# ====================================================================
# Parte 2: CONFIGURAÇÕES GLOBAIS
# ====================================================================
print("\nDefinindo configurações globais...")
TICKER = 'VALE3.SA'
START_DATE = '2015-01-01'
END_DATE = date.today().strftime("%Y-%m-%d")
SEQ_LENGTH = 60
FUTURE_DAYS = 5
THRESHOLD = 0.03
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 40
BASE_DIR = '/content/stock_data_hybrid'

if os.path.exists(BASE_DIR):
    shutil.rmtree(BASE_DIR)
os.makedirs(BASE_DIR, exist_ok=True)
print("Configurações definidas e pastas criadas.")


# ====================================================================
# Parte 3: DOWNLOAD E CÁLCULO MANUAL DE INDICADORES (Using Pandas Series)
# ====================================================================
print(f"\nBaixando dados históricos para {TICKER}...")
data = yf.download(TICKER, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)
data.dropna(inplace=True)

print("Calculating technical indicators manually using Pandas Series...")

# Calculate Bollinger Bands directly on pandas Series
window = 20
data['volatility_bbm'] = data['Close'].rolling(window=window).mean()
data['volatility_bbh'] = data['Close'].rolling(window=window).mean() + data['Close'].rolling(window=window).std() * 2
data['volatility_bbl'] = data['Close'].rolling(window=window).mean() - data['Close'].rolling(window=window).std() * 2

# Calculate RSI directly on pandas Series using EMA
window_rsi = 14
delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

avg_gain = gain.ewm(com=window_rsi - 1, adjust=False).mean()
avg_loss = loss.ewm(com=window_rsi - 1, adjust=False).mean()

# Avoid division by zero and handle potential inf values
rs = avg_gain / avg_loss
rs = rs.replace([np.inf, -np.inf], np.nan).fillna(0) # Replace inf with nan, then fill nan with 0

data['momentum_rsi'] = 100 - (100 / (1 + rs))


data.dropna(inplace=True)
print(f"{len(data)} days of data processed with indicators.")


# ====================================================================
# Parte 4: GERAÇÃO DAS IMAGENS E ALINHAMENTO DOS DADOS (CORRIGIDO)
# ====================================================================
print("\nGerando imagens e alinhando dados para o modelo híbrido...")

features_lstm = ['Close', 'Volume', 'volatility_bbh', 'volatility_bbl', 'volatility_bbm', 'momentum_rsi']
# O scaler é o mesmo, não precisa mudar
scaler = MinMaxScaler(feature_range=(0, 1)) # Moved scaler definition here
# data[features_lstm] = scaler.fit_transform(data[features_lstm])

# Criamos um novo dataframe para os dados normalizados para não afetar os cálculos de rótulo
data_normalized = data.copy()
data_normalized[features_lstm] = scaler.fit_transform(data[features_lstm])


image_paths, lstm_sequences, labels = [], [], []
class_names = ['ALTA', 'BAIXA', 'NEUTRA']
class_map = {name: i for i, name in enumerate(class_names)}

for i in range(len(data) - SEQ_LENGTH - FUTURE_DAYS):
    # --- Define o Rótulo (usando os dados originais, não normalizados) ---
    preco_final_janela = data['Close'].iloc[i + SEQ_LENGTH - 1]
    preco_futuro = data['Close'].iloc[i + SEQ_LENGTH + FUTURE_DAYS - 1]

    label_str = 'NEUTRA'
    # AJUSTE 1: A comparação agora é feita diretamente com os números, usando .iloc[0]
    if preco_futuro.iloc[0] > preco_final_janela.iloc[0] * (1 + THRESHOLD):
        label_str = 'ALTA'
    elif preco_futuro.iloc[0] < preco_final_janela.iloc[0] * (1 - THRESHOLD):
        label_str = 'BAIXA'
    labels.append(class_map[label_str])

    # --- Gera e Salva a Imagem (usando os dados normalizados) ---
    dados_janela_normalizados = data_normalized.iloc[i : i + SEQ_LENGTH] # Usar dados normalizados para a imagem/LSTM
    nome_arquivo = f"img_{i}.png"
    caminho_completo = os.path.join(BASE_DIR, nome_arquivo)

    fig, ax = plt.subplots(3, 1, figsize=(IMG_SIZE/100, IMG_SIZE/100), dpi=100, gridspec_kw={'height_ratios': [2, 1, 1]})
    ax[0].plot(dados_janela_normalizados['Close'].values, color='black')
    ax[0].plot(dados_janela_normalizados['volatility_bbh'].values, color='blue', alpha=0.5)
    ax[0].plot(dados_janela_normalizados['volatility_bbl'].values, color='blue', alpha=0.5)
    ax[1].plot(dados_janela_normalizados['momentum_rsi'].values, color='purple')
    ax[2].bar(range(len(dados_janela_normalizados)), dados_janela_normalizados['Volume'].values.flatten(), color='gray')
    for axis in ax: axis.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0.01)
    fig.savefig(caminho_completo, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # --- Guarda os dados alinhados (usando os dados normalizados) ---
    image_paths.append(caminho_completo)
    lstm_sequences.append(dados_janela_normalizados[features_lstm].values)

# --- Conversão e Divisão dos Dados ---
image_paths = np.array(image_paths)
lstm_sequences = np.array(lstm_sequences)
# AJUSTE 2: Mude para np.int32 para que o Keras aceite os rótulos!
labels = np.array(labels, dtype=np.int32)

print(f"Total de {len(labels)} exemplos gerados e alinhados.")

# O restante do código de divisão de dados permanece o mesmo
img_train_paths, img_val_paths, X_train_lstm, X_val_lstm, y_train, y_val = train_test_split(
    image_paths, lstm_sequences, labels, test_size=0.2, random_state=42, stratify=labels
)
print(f"Dados divididos em {len(y_train)} para treino e {len(y_val)} para validação.")


# ====================================================================
# Parte 5: CONSTRUÇÃO DO MODELO HÍBRIDO (API FUNCIONAL)
# ====================================================================
print("\nConstruindo o Cérebro Híbrido (CNN + LSTM)...")

input_cnn = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='entrada_imagem')
cnn_branch = Rescaling(1./255)(input_cnn)
cnn_branch = Conv2D(32, 3, activation='relu')(cnn_branch)
cnn_branch = MaxPooling2D()(cnn_branch)
cnn_branch = Conv2D(64, 3, activation='relu')(cnn_branch)
cnn_branch = MaxPooling2D()(cnn_branch)
cnn_branch = Flatten()(cnn_branch)
cnn_branch = Dense(64, activation='relu')(cnn_branch)

input_lstm = Input(shape=(SEQ_LENGTH, len(features_lstm)), name='entrada_numerica')
lstm_branch = LSTM(units=50)(input_lstm)
lstm_branch = Dropout(0.2)(lstm_branch)
lstm_branch = Dense(25, activation='relu')(lstm_branch)

fused = concatenate([cnn_branch, lstm_branch])
final_head = Dense(64, activation='relu')(fused)
final_head = Dropout(0.5)(final_head)
final_output = Dense(len(class_names), activation='softmax', name='saida_final')(final_head)

hybrid_model = Model(inputs=[input_cnn, input_lstm], outputs=final_output, name="Cerebro_Hibrido")
hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
hybrid_model.summary()


# ====================================================================
# Parte 6: GERADOR DE DADOS E TREINAMENTO
# ====================================================================

# O gerador é definido aqui, de forma limpa e correta
def data_generator(image_paths, lstm_sequences, labels, batch_size):
    num_samples = len(image_paths)
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        for offset in range(0, num_samples, batch_size):
            batch_indices = indices[offset:offset+batch_size]

            batch_images = np.array([img_to_array(load_img(p, target_size=(IMG_SIZE, IMG_SIZE))) for p in image_paths[batch_indices]])
            batch_lstm = lstm_sequences[batch_indices]
            batch_labels = labels[batch_indices] # Directly use the numerical labels

            print(f"Type of batch_labels before yielding: {type(batch_labels)}")
            print(f"Dtype of batch_labels before yielding: {batch_labels.dtype}")
            print(f"Shape of batch_labels before yielding: {batch_labels.shape}")


            yield ({'entrada_imagem': batch_images, 'entrada_numerica': batch_lstm},
                   batch_labels) # Yield the numerical labels as a single array


# Criando os geradores para treino e validação
train_gen = data_generator(img_train_paths, X_train_lstm, y_train, BATCH_SIZE)
val_gen = data_generator(img_val_paths, X_val_lstm, y_val, BATCH_SIZE)

print("\nIniciando o treinamento do Cérebro Híbrido...")
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1)

history_hybrid = hybrid_model.fit(
    train_gen,
    steps_per_epoch=len(y_train) // BATCH_SIZE,
    validation_data=val_gen,
    validation_steps=len(y_val) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)


# ====================================================================
# Parte 7: AVALIAÇÃO E PREVISÃO FINAL
# ====================================================================
print("\nFazendo previsão final com o Cérebro Híbrido...")
# Preparar os dados mais recentes para a previsão
dados_recentes_nao_normalizados = yf.download(TICKER, period=f"{SEQ_LENGTH+20}d", auto_adjust=True, progress=False) # +20 para garantir dados para indicadores
dados_recentes_nao_normalizados.dropna(inplace=True)

# Recalculate indicators for recent data using Pandas Series
window = 20
dados_recentes_nao_normalizados['volatility_bbm'] = dados_recentes_nao_normalizados['Close'].rolling(window=window).mean()
dados_recentes_nao_normalizados['volatility_bbh'] = dados_recentes_nao_normalizados['Close'].rolling(window=window).mean() + dados_recentes_nao_normalizados['Close'].rolling(window=window).std() * 2
dados_recentes_nao_normalizados['volatility_bbl'] = dados_recentes_nao_normalizados['Close'].rolling(window=window).mean() - dados_recentes_nao_normalizados['Close'].rolling(window=window).std() * 2

window_rsi = 14
delta_recente = dados_recentes_nao_normalizados['Close'].diff()
gain_recente = delta_recente.where(delta_recente > 0, 0)
loss_recente = -delta_recente.where(delta_recente < 0, 0)

avg_gain_recente = gain_recente.ewm(com=13, adjust=False).mean()
avg_loss_recente = loss_recente.ewm(com=13, adjust=False).mean()

rs_recente = avg_gain_recente / avg_loss_recente
rs_recente = rs_recente.replace([np.inf, -np.inf], np.nan).fillna(0)

dados_recentes_nao_normalizados['momentum_rsi'] = 100 - (100 / (1 + rs_recente))


dados_recentes_nao_normalizados.dropna(inplace=True)


dados_recentes_normalizados = dados_recentes_nao_normalizados.copy()
dados_recentes_normalizados[features_lstm] = scaler.transform(dados_recentes_nao_normalizados[features_lstm])
dados_para_previsao = dados_recentes_normalizados.tail(SEQ_LENGTH)

# Input Numérico
X_pred_lstm = dados_para_previsao[features_lstm].values
X_pred_lstm_expanded = np.expand_dims(X_pred_lstm, 0)

# Input Visual
nome_arquivo_pred = "pred_final.png"
caminho_pred = os.path.join(BASE_DIR, nome_arquivo_pred)
fig, ax = plt.subplots(3, 1, figsize=(IMG_SIZE/100, IMG_SIZE/100), dpi=100, gridspec_kw={'height_ratios': [2, 1, 1]})
ax[0].plot(dados_para_previsao['Close'].values, color='black')
ax[0].plot(dados_para_previsao['volatility_bbh'].values, color='blue', alpha=0.5)
ax[0].plot(dados_para_previsao['volatility_bbl'].values, color='blue', alpha=0.5)
ax[1].plot(dados_para_previsao['momentum_rsi'].values, color='purple')
ax[2].bar(range(len(dados_para_previsao)), dados_para_previsao['Volume'].values.flatten(), color='gray')
for axis in ax: axis.axis('off')
plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0.01)
fig.savefig(caminho_pred, bbox_inches='tight', pad_inches=0)
plt.close(fig)

img_pred = load_img(caminho_pred, target_size=(IMG_SIZE, IMG_SIZE))
img_pred_array = np.expand_dims(img_to_array(img_pred), 0)

# Fazer a previsão com os dois inputs
prediction = hybrid_model.predict({'entrada_imagem': img_pred_array, 'entrada_numerica': X_pred_lstm_expanded})
score = prediction[0]

print("\n" + "="*60)
print(f"🔮 PREDIÇÃO HÍBRIDA PARA OS PRÓXIMOS {FUTURE_DAYS} DIAS DE {TICKER}")
print("="*60)
for i, class_name in enumerate(class_names):
    print(f"Probabilidade de {class_name}: {100 * score[i]:.2f}%")

predicted_class_index = np.argmax(score)
predicted_class = class_names[predicted_class_index]
print(f"\n🟢 TENDÊNCIA MAIS PROVÁVEL: {predicted_class}")
print("="*60)

Instalando e importando bibliotecas...
Bibliotecas carregadas com sucesso!

Definindo configurações globais...
Configurações definidas e pastas criadas.

Baixando dados históricos para VALE3.SA...
Calculating technical indicators manually using Pandas Series...
2606 days of data processed with indicators.

Gerando imagens e alinhando dados para o modelo híbrido...
Total de 2541 exemplos gerados e alinhados.
Dados divididos em 2032 para treino e 509 para validação.

Construindo o Cérebro Híbrido (CNN + LSTM)...


Model: "Cerebro_Hibrido"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ entrada_imagem      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 128, 128,  │          0 │ entrada_imagem[0… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 126, 126,  │        896 │ rescaling_3[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 63, 63,    │          0 │ conv2d_6[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 61, 61,    │     18,496 │ max_pooling2d_6[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ entrada_numerica    │ (None, 60, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 30, 30,    │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 50)        │     11,400 │ entrada_numerica… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 57600)     │          0 │ max_pooling2d_7[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 50)        │          0 │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 64)        │  3,686,464 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 25)        │      1,275 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 89)        │          0 │ dense_9[0][0],    │
│ (Concatenate)       │                   │            │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 64)        │      5,760 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 64)        │          0 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ saida_final (Dense) │ (None, 3)         │        195 │ dropout_7[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,724,486 (14.21 MB)

 Trainable params: 3,724,486 (14.21 MB)

 Non-trainable params: 0 (0.00 B)


Iniciando o treinamento do Cérebro Híbrido...
Type of batch_labels before yielding: <class 'numpy.ndarray'>
Dtype of batch_labels before yielding: int32
Shape of batch_labels before yielding: (32,)
Type of batch_labels before yielding: <class 'numpy.ndarray'>
Dtype of batch_labels before yielding: int32
Shape of batch_labels before yielding: (32,)
Epoch 1/40
 1/63 ━━━━━━━━━━━━━━━━━━━━ 2:49 3s/step - accuracy: 0.3125 - loss: 1.1033Type of batch_labels before yielding: <class 'numpy.ndarray'>
Dtype of batch_labels before yielding: int32
Shape of batch_labels before yielding: (32,)
 2/63 ━━━━━━━━━━━━━━━━━━━━ 26s 428ms/step - accuracy: 0.3359 - loss: 1.3472Type of batch_labels before yielding: <class 'numpy.ndarray'>
Dtype of batch_labels before yielding: int32
Shape of batch_labels before yielding: (32,)
 3/63 ━━━━━━━━━━━━━━━━━━━━ 25s 417ms/step - accuracy: 0.3420 - loss: 1.5502Type of batch_labels before yielding: <class 'numpy.ndarray'>
Dtype of batch_labels before yielding: int32
Shap